# Sugestão de cidade utilizando o Índice de Criminalidade do Estado do Rio Grande do Sul
### Para o trabalho final da disciplina de Sistemas de Recomendação foi escolhido os bancos de dados Índice de Criminalidade (RS, 2023) e Estimativas Populacionais (RS, 2022). Os dados são reais e estão disponíveis no portal Dados Abertos RS do Estado do Rio Grande do Sul.

## Pré-processamento
### Houve a necessidade de realizar o pré-processamento dos dados, pois foram disponibilizados em planilhas Excel com formatações diversas.

In [136]:
import pandas as pd

#### Abertura dos arquivos de criminalidade por cidade em cada mês.

In [137]:
# Declarando os meses
meses = ['janeiro', 'fevereiro', 'marco', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

dfs = []

# Lendo os arquivos conforme os meses e adicionando a lista de dataframes
for mes in meses:
        arquivo = pd.read_csv(f'data\\tocsv\\{mes}.csv', delimiter=';')
        dfs.append(arquivo)

# Concatenando todos os dataframes ignorando o index
df = pd.concat(dfs, ignore_index=True)

# Agrupando por município e ibge e somando os valores
df_criminal = df.groupby(['municipios', 'ibge']).sum().reset_index()

# Salvando o dataframe em um arquivo csv
df.to_csv('data\\tocsv\\total.csv', sep=';', index=False)

df_criminal

,municipios,ibge,homicidio_doloso,total_vitimas_homicidio_doloso,latrocinio,furtos,abigeato,furto_veiculo,roubos,roubo_veiculo,estelionato,delitos_armas_municoes,entorpecente_posse,entorpecente_trafico,vitimas_latrocinio,vitimas_lesao_corporal_morte,total_vitimas_crimes_violentos
0,acegua,4300034,0,0,0,20,8,0,3,0,27,4,3,0,0,0,0
1,agua santa,4300059,1,1,0,20,3,4,1,0,13,2,0,0,0,0,2
2,agudo,4300109,1,1,0,102,2,2,3,1,51,3,7,10,0,0,1
3,ajuricaba,4300208,0,0,0,26,1,0,1,0,28,2,3,0,0,0,0
4,alecrim,4300307,1,1,0,36,10,0,2,0,17,10,0,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,vista alegre do prata,4323606,0,0,0,3,0,0,0,0,8,3,0,0,0,0,0
493,vista gaucha,4323705,1,1,0,11,0,0,0,0,6,2,0,0,0,0,1
494,vitoria das missoes,4323754,0,0,0,20,3,0,0,0,7,1,4,0,0,0,0
495,westfalia,4323770,0,0,0,10,2,1,0,0,20,1,1,0,0,0,0


#### Abertura do arquivo de quantidade de população por cidade em cada ano.

In [138]:
# Lendo o arquivo de população
df_pop = pd.read_csv('data\\tocsv\\populacao.csv', delimiter=';')

#df_pop.info() mostra que as colunas de 4 em diante são do tipo object e para fazer operações matemáticas é necessário converter para inteiro

# Substituindo os valores de '.' e '-' por 0 e convertendo para inteiro
for coluna in df_pop.columns[4:]:
    df_pop[coluna] = df_pop[coluna].str.replace('.', '').str.replace('-', '0').astype(int)

df_pop.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   municipios  497 non-null    object 
 1   ibge        497 non-null    int64  
 2   latitude    497 non-null    float64
 3   longitude   497 non-null    float64
 4   2010        497 non-null    int32  
 5   2011        497 non-null    int32  
 6   2012        497 non-null    int32  
 7   2013        497 non-null    int32  
 8   2014        497 non-null    int32  
 9   2015        497 non-null    int32  
 10  2016        497 non-null    int32  
 11  2017        497 non-null    int32  
 12  2018        497 non-null    int32  
 13  2019        497 non-null    int32  
 14  2020        497 non-null    int32  
 15  2021        497 non-null    int32  
dtypes: float64(2), int32(12), int64(1), object(1)
memory usage: 39.0+ KB


### União dos dois banco de dados

In [127]:
# O merge é feito com base na coluna ibge como forma de chave estrangeira com o método inner
df_merged = pd.merge(df_criminal, df_pop, how = 'inner', on = 'ibge')

df_merged.info()

df_merged


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   municipios_x                    497 non-null    object 
 1   ibge                            497 non-null    int64  
 2   homicidio_doloso                497 non-null    int64  
 3   total_vitimas_homicidio_doloso  497 non-null    int64  
 4   latrocinio                      497 non-null    int64  
 5   furtos                          497 non-null    int64  
 6   abigeato                        497 non-null    int64  
 7   furto_veiculo                   497 non-null    int64  
 8   roubos                          497 non-null    int64  
 9   roubo_veiculo                   497 non-null    int64  
 10  estelionato                     497 non-null    int64  
 11  delitos_armas_municoes          497 non-null    int64  
 12  entorpecente_posse              497 

,municipios_x,ibge,homicidio_doloso,total_vitimas_homicidio_doloso,latrocinio,furtos,abigeato,furto_veiculo,roubos,roubo_veiculo,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,acegua,4300034,0,0,0,20,8,0,3,0,...,4539,4520,4564,4483,4472,4412,4487,4516,4540,4505
1,agua santa,4300059,1,1,0,20,3,4,1,0,...,3858,3898,3959,3922,3977,4013,4057,4107,4093,4256
2,agudo,4300109,1,1,0,102,2,2,3,1,...,16731,16838,16851,16701,16595,16475,16537,16556,16760,16612
3,ajuricaba,4300208,0,0,0,26,1,0,1,0,...,7389,7431,7299,7241,7279,7325,7546,7485,7584,7447
4,alecrim,4300307,1,1,0,36,10,0,2,0,...,7074,6891,6814,6598,6594,6569,6513,6435,6301,6403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,vista alegre do prata,4323606,0,0,0,3,0,0,0,0,...,1578,1539,1582,1630,1677,1645,1648,1704,1721,1746
493,vista gaucha,4323705,1,1,0,11,0,0,0,0,...,2842,2828,2787,2790,2802,2834,2885,2940,2987,3002
494,vitoria das missoes,4323754,0,0,0,20,3,0,0,0,...,3453,3403,3415,3448,3383,3439,3389,3438,3397,3405
495,westfalia,4323770,0,0,0,10,2,1,0,0,...,2864,2957,2974,3007,3039,3088,3136,3125,3226,3257


## Cálculo da taxa de criminalidade a cada 1000 mil habitantes.

In [134]:
# Lista das colunas que precisam ser calculadas
colunas_taxa = df_criminal.columns[2:]

# Calculando a taxa de criminalidade para cada mês
for coluna_taxa in colunas_taxa:
    for coluna_pop in df_pop.columns[4:]:
            
            # Calculando a taxa de criminalidade por 100.000 habitantes
            df_merged[f'taxa_{coluna_taxa}'] = df_criminal[coluna_taxa] / df_pop[coluna_pop] * 100000

df_merged
    

,municipios_x,ibge,homicidio_doloso,total_vitimas_homicidio_doloso,latrocinio,furtos,abigeato,furto_veiculo,roubos,roubo_veiculo,...,taxa_furto_veiculo,taxa_roubos,taxa_roubo_veiculo,taxa_estelionato,taxa_delitos_armas_municoes,taxa_entorpecente_posse,taxa_entorpecente_trafico,taxa_vitimas_latrocinio,taxa_vitimas_lesao_corporal_morte,taxa_total_vitimas_crimes_violentos
0,acegua,4300034,0,0,0,20,8,0,3,0,...,0.000000,66.592675,0.000000,599.334073,88.790233,66.592675,0.000000,0.0,0.0,0.000000
1,agua santa,4300059,1,1,0,20,3,4,1,0,...,93.984962,23.496241,0.000000,305.451128,46.992481,0.000000,0.000000,0.0,0.0,46.992481
2,agudo,4300109,1,1,0,102,2,2,3,1,...,12.039490,18.059234,6.019745,307.006983,18.059234,42.138213,60.197448,0.0,0.0,6.019745
3,ajuricaba,4300208,0,0,0,26,1,0,1,0,...,0.000000,13.428226,0.000000,375.990332,26.856452,40.284678,0.000000,0.0,0.0,0.000000
4,alecrim,4300307,1,1,0,36,10,0,2,0,...,0.000000,31.235358,0.000000,265.500547,156.176792,0.000000,62.470717,0.0,0.0,15.617679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,vista alegre do prata,4323606,0,0,0,3,0,0,0,0,...,0.000000,0.000000,0.000000,458.190149,171.821306,0.000000,0.000000,0.0,0.0,0.000000
493,vista gaucha,4323705,1,1,0,11,0,0,0,0,...,0.000000,0.000000,0.000000,199.866755,66.622252,0.000000,0.000000,0.0,0.0,33.311126
494,vitoria das missoes,4323754,0,0,0,20,3,0,0,0,...,0.000000,0.000000,0.000000,205.580029,29.368576,117.474302,0.000000,0.0,0.0,0.000000
495,westfalia,4323770,0,0,0,10,2,1,0,0,...,30.703101,0.000000,0.000000,614.062020,30.703101,30.703101,0.000000,0.0,0.0,0.000000
